In [8]:
def source_fromcode(txt, k):
    src = {}
    total_chars = 0
    for i in range(len(txt)-k+1):
        substr = txt[i:i+k]
        if substr in src:
            src[substr] += 1
        else:
            src[substr] = 1
        total_chars += 1

    return (sorted(src.items()), total_chars)

In [9]:
def dec2binary(a, k):
    binary = bin(a)[2:]
    padded_binary = '0'*(k-len(binary)) + binary  
    return padded_binary


In [10]:
import math


def arithmetic_encode(txt,k,src,n):
    src = dict(src)
    keys = list(src.keys())
    total = 0
    for clave, valor in src.items():
        total += valor
    a = 0
    b = 2**k-1
    print("Interval inicial", a, b)
    print()
    c = ''
    count = 0

    for i in range(0, len(txt), n):
        lletra = txt[i:i+n]
        d = b-a+1
        it = keys.index(lletra)
        p = 0
        while(it > 0):
            p += src[keys[it-1]]
            it -= 1
        b = a + math.floor(d * (p + src[lletra])/total)-1
        a = a + math.floor(d * p/total)    

        print("Es codifica la lletra:", lletra, " i queda el subinterval", a, b)
        a_bin = dec2binary(a, k)
        b_bin = dec2binary(b, k)
        print("En binari:", a_bin, b_bin)

        while (a_bin[0] == b_bin[0]):
            char = a_bin[0]
            c += char
            a = a*2
            b = b*2 +1
            if (char == '1'):
                a = a - 2**k
                b = b - 2**k
            a_bin = dec2binary(a, k)
            b_bin = dec2binary(b, k)
            print ("Reescalat:", a_bin, b_bin)
            print("codi:", c)
            if count > 0:
                print("Afegits", count, "bits d'underflow")
                if char == '0':
                    char = '1'
                else:
                    char = '0'
                while(count > 0):
                    c += char
                    count -= 1
                print("codi:", c)

        if a_bin[1] == '1' and b_bin[1] == '0':
            while(not(a_bin[0:2] == '00' or b_bin[0:2] == '11')):
                count += 1
                a = a*2-2**(k-1)
                b = b*2-2**(k-1)+1
                a_bin = dec2binary(a, k)
                b_bin = dec2binary(b, k)
                print("Underflow num.", count)
                print ("Reescalat:", a_bin, b_bin)
        print("Interval ha quedat", a, b)
        print()
    c += '1'
    print("afegim un 1 al final i queda el codi", c)
    return c

In [11]:
def arithmetic_decode(code, k, src, l):
    x = ''
    a = 0
    b = 2**k-1
    count = k-1
    end = False
    y = int(code[0:k], 2)
    total = 0
    for clave, valor in src.items():
        total += valor

    print("Alpha, gamma i beta inicials:", a, y, b)
    while l != len(x):
        p = 0
        aux_p = 0
        d = b-a+1
        for clave in src.keys():
            aux_p = p
            p += src[clave]/total
            if a+math.floor(d * p) > y:
                print(clave)
                x += clave
                break
        print("Determinen nova lletra:", x)
        b = a + math.floor(d * p)-1
        a = a + math.floor(d * aux_p) 
        print("Nous alpha, gamma i beta:", a, y, b)
        a_bin = dec2binary(a, k)
        y_bin = dec2binary(y, k)
        b_bin = dec2binary(b, k)
        print("En binari:", a_bin, y_bin, b_bin)

        while (a_bin[0] == b_bin[0]):
            if count < len(code)-1:
                count += 1
            else:
                end = True
            char = a_bin[0]
            a = a*2
            b = b*2 +1
            if code[count] == '0' or end:
                y = y*2
            else:
                y = y*2+1
            if (char == '1'):
                a = a - 2**k
                b = b - 2**k
                y = y - 2**k
            a_bin = dec2binary(a, k)
            y_bin = dec2binary(y, k)
            b_bin = dec2binary(b, k)
            print ("Reescalat:", a_bin, y_bin, b_bin)
            
        if a_bin[1] == '1' and b_bin[1] == '0':
            while(not(a_bin[0:2] == '00' or b_bin[0:2] == '11')):
                if count < len(code)-1:
                    count += 1
                else:
                    end = True
                a = a*2-2**(k-1)
                b = b*2-2**(k-1)+1
                if code[count] == '0' or end:
                    y = y*2-2**(k-1)
                else:
                    y = y*2-2**(k-1)+1
                a_bin = dec2binary(a, k)
                y_bin = dec2binary(y, k)
                b_bin = dec2binary(b, k)
                print ("Underflow:", a_bin, y_bin, b_bin)
        
        print("Queden alpha, gamma i beta:", a, y, b)
        print()
    return x

In [18]:
C = "010110101100011110010111100000001101001101001010011001100110011001000111100110010000011111111110001000011001001101000001010000101111110100010100011000000011110001101011111101110101001000010011001101111101001000010000001100111110110101011000000111011001010110111001110111000111101000010001110011001000000011111111111110111100010000011111101010010011110111001000010110011010001110111101100101111001110111011111001111001000010000001100010010100010000110100100010001101100001100101001110010011011011110111010111111110001111001011110100100001111000111110111010010100000011110101010000110100000011011100001100101001000111110010111010111001101010011101010110101010000010111100101100000000001100011101100010011000111011110100000001101010010010000001110010110110101000010110011000010110100010100111100001100101101110101000111100000110110111010100010000110111101010001100100110001000001110001001101010110100110111011100000111010010110110001111110100011111011110011110000001101011010001110100111100001011101001100001111011001110011010100001010011100111010011010101000100000111111001111110111001011001010010100101111111111110100110110011100000001010100110111111100011110110011111000000010111111011001001110111100001011100011001010110000110111110011111011111010001010100011011000111110000110110011001010001010001001011111000001100010110101011000110110010010110110010000111110111110011001111001110101101011111001011011010111111111111100100100010000000101111111000000011100010101000010011110100101011001000110010001011001100111101000011011100011110110011101101001010011011001011111011110111110011011010010111101111001011110101101001010110000100011010111010111100010010110111111110100011101001011110110001100101110110001001111010010000100100101001111110110001000001011000100111010111110100000011011010110110101010001110100111010101000001111110011100110001100110110011100001100100100001010001001101110110011101101100000010111101010010011010110001100001100011110001000011111001110010100001100000100110001101110001001011100010101111100000001001100100000110000101111110001101100110101001010001001100111100011010101110001100111001100000100111111111001110001101010110001000000001101011101011100010000101001111110010111011101011000100100111111011101100110101000010001101010000001011001001100101010101010100101110111110001010010000111100000011110111001000111100110101000011000011100110001110111110111000111000111111110101000110010011001101110101000010110111111010111101001000101001000011001110100100110010001000110101010111110111000000010110101110000101110011011101111110011001000101001110001110001010101010001010000101011100111010110000011010110100101000010110000101111111010010010010100010011001110111111000110011010101100100111000001100001000011100010101010110101011010110000011100101110110101100010000101110110110011011111011101001111110101111011110011001101100111001010110000001100010101110101010100110110101000100011010011110001101010110110000110111111101000100110111001110010001011010111111000000100110001010011110110110101110010011010010011011101010111001111110011010101100111101101000000100000110110000100100001100101011001110000101010010010101011110000101100010111111111110101001101111011110001010111100001110111110011011100000000010100011100010100111011100001100101100001010110110110100010110011111100011000001100111101000100100001101101000100111010111110101010100100110100010000000101010000111111100010001101001001001110001111111000011001101000010111001111011100000101100011101110111000100110011000000111101110000110000000011110000010101001001110010100000010111000101010101100110000101010110110111000110111011110010000111000010010101001111001100111001011110011011111011011000100011011111011011110101001110001011010101111110000101110000010101000101001010111000001000010010001000111010111111000010010000010101000001100111011010101011000011111011000101010111000000001111001111001000100010001101101000001110010000111110111101000100110001110011010100111010011010100011011010010001101011001100110000101000011100000100000000010101001100110011111100110111011101101010101011100101010011100101000000101100011111101011111001001011001111101110100101110100001100001000100110111101011110001111001110110100010001000101010000011100100001010110100011101111010111011011001010101110110100110101100111110111010111001110111100000100011001011111100011000110111001100100000101111000001000100011000101011111010001100110101100000111100001100101011101000001110011001100010000001110001010010001000000001001110000101010000101001110100100100000101011001101110011110000000011100000010110010111000010000111100111100111110100101110100001111001010110000101111011001010101010111101010011100010000011001001111011011010111110011001000011011001100010111101001100000100000010101000011110100011001000101101000110111010011111101100111110100110101000110000110101101111111000010011001111111001111011000110011011110101100111111000100101011001011011000100000011011111000010101011110101011101010101001010001101110011101101100000001001001"
src =  [('00', 1872), ('01', 262), ('10', 16), ('11', 241)]
k = 15
#n = 2
src = source_fromcode('abracadabra', 1)

# = arithmetic_encode(txt, k, src, n)
print(src)

print(arithmetic_encode('abracadabra', 15, src, 1))

#E = arithmetic_decode(C, k, dict(src), 9794) 
#print(E == txt)

([('a', 5), ('b', 2), ('c', 1), ('d', 1), ('r', 2)], 11)


ValueError: dictionary update sequence element #0 has length 5; 2 is required